In [1]:
###############
# 1. import
#####

import pandas as pd
from datetime import datetime, timedelta
import re
import time 
from bs4 import BeautifulSoup

# local only
import os
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import cx_Oracle as oci

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)


In [2]:
###############
# 2.자료 추출
#####

# DB연결
conn = pymysql.connect(host = '**.***.*.*', user ='__USER_ID__', password = "__PASSWORD__", database='__DATABASE__')

# cursor 설정
cursor = conn.cursor(pymysql.cursors.DictCursor)

#자료 불러오기 : 밀크세라마이드 최근 7주
sql= '''select * from (
            select (SELECT CODE_NM FROM JT_CODE B WHERE A.COMPANY_CODE = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') as 수집체널
            , GROUP_CONCAT( distinct (SELECT CODE_NM FROM JT_CODE B WHERE A.PRODUCT_CODE = B.CODE AND B.CODE_GRP_ID = 'PRODUCT') SEPARATOR ', ') as 구매제품
            , A.WRITER as "작성자ID" , A.CONTENTS as 내용 , A.WRITE_DT as 작성일자 , A.GRADE as "평점(5점 만점)"
            FROM MAEIL_REVIEW A
            where WRITE_DT between DATE_SUB(CURDATE(), INTERVAL 7 DAY) and  DATE_SUB(CURDATE(), INTERVAL 1 DAY) 
            and A.COMPANY_CODE in (13261, 13358)
            and PRODUCT_CODE not like 'G_%'
            and PRODUCT_CODE = '106213' /* 밀크세라마이드 */
            group by A.COMPANY_CODE, A.WRITER, A.CONTENTS, A.WRITE_DT, A.GRADE
            order by A.WRITE_DT asc) A
        order by LENGTH(내용) desc
        '''
cursor.execute(sql)
df1 = cursor.fetchall()
df1 = pd.DataFrame(df1)
df2 = df1.head(5)

In [3]:
df1.tail(2)

,수집체널,구매제품,작성자ID,내용,작성일자,평점(5점 만점)
51,3 네이버,셀렉스 밀크세라마이드 [ 요거트맛 ] [ 2.7G X 28 ],dogn****,잘 먹어보겠습니다~,20210317,5
52,3 네이버,셀렉스 밀크세라마이드 [ 요거트맛 ] [ 2.7G X 28 ],gots****,잘 받았습니다~^^,20210318,5


In [4]:
# -*- coding=utf-8 -*-
# os.environ['NLS_LANG'] = '.UTF8'

# 접속정보
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

# query 열기
f = open("C:/Users/MAEIL/OneDrive - maeil365/auto_send/oracel_query/selex_review_1.txt", 'r')
line = f.read()
f.close()

# ## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()
df3 = pd.DataFrame(df3, columns = ['수집체널', '구매제품', '작성자ID', '내용', '작성일자', '평점(5점 만점)'])

In [5]:
df3.head()

,수집체널,구매제품,작성자ID,내용,작성일자,평점(5점 만점)
0,selexmall,밀크세라마이드 콜라겐 마스크 4박스 (25gx20매입),ksr9139,오늘 받아서 처음 사용해봤어요. 사용후 약간의 끈적임이 있긴하지만 일단은 햅격입니다...,20210317,4
1,selexmall,밀크세라마이드 콜라겐 1박스 (2.7gx28포),ksr9139,오늘 받아서 한포 먹어봤어요. 뜯어보니 안에 분말이 조금 꾸덕한 느낌으로 포지안쪽에...,20210317,4
2,selexmall,[ID당 1회 구매]밀크세라마이드 콜라겐 1주일 체험키트 (2.7gx7포),miyeonjun,요거트 맛이라서 달달한 맛이에요. 가루도 고운편고. 괜챦은것 같아요,20210317,5
3,selexmall,[ID당 1회 구매]밀크세라마이드 콜라겐 1주일 체험키트 (2.7gx7포),bobbe,몸에 좋을것 같아서 구매해 봤어요ㅎ\n꾸준히 먹어 보려구요 만족합니다~^^,20210317,5
4,selexmall,[ID당 1회 구매]밀크세라마이드 콜라겐 1주일 체험키트 (2.7gx7포),sms0212,먹던 콜라겐이 있기는한데 믿을수있는 회사라 주문했습니다. 코어프로틴과함께 건강하게 ...,20210317,5


In [8]:
df0 = pd.concat([df1, df3],  ignore_index=True )
df0['작성일자'] = df0['작성일자'].str[2:4] + '/'+ df0['작성일자'].str[4:6] + '/' + df0['작성일자'].str[6:8]

In [28]:
df_count = pd.crosstab(df0.수집체널, df0.작성일자)

In [29]:
df_count

작성일자,21/03/12,21/03/13,21/03/14,21/03/15,21/03/16,21/03/17,21/03/18
수집체널,,,,,,,
3 네이버,12,6,1,9,6,9,10
selexmall,11,6,1,5,7,9,0
